In [1]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import when

In [2]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("dbcreation_alkemy")\
.getOrCreate()

22/12/08 17:23:46 WARN Utils: Your hostname, MacBook-Pro-di-Fabiana.local resolves to a loopback address: 127.0.0.1; using 192.168.1.52 instead (on interface en0)
22/12/08 17:23:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/08 17:23:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/08 17:23:48 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
regular = spark.read.csv('clicks_regular_all.csv', header=True, inferSchema=True)
bidding = spark.read.csv('clicks_bidding_all.csv', header=True, inferSchema=True)
concatenato = regular.union(bidding)

In [4]:
concatenato = concatenato.withColumn("type", when(concatenato.type == "Regular",0) \
      .otherwise(1))

concatenato = concatenato.withColumn("position", when(concatenato.type == 0,concatenato.position + 3) \
      .otherwise(concatenato.position))

concatenato = concatenato.where(
    ((concatenato.position != 0)
     & (concatenato.type != 0))
)

In [5]:
concatenato.show()

+-------------------+------+--------+---------+---------+-------+----+----------+
|               date|seller|position|price_max|price_min|  price|type|product_id|
+-------------------+------+--------+---------+---------+-------+----+----------+
|2021-04-02 11:48:02|    48|     1.0|  25990.0|  21990.0|25990.0|   1|    140476|
|2021-04-02 11:48:04|   180|     1.0|  51990.0|  42989.0|45490.0|   1|    123606|
|2021-04-02 11:48:15|    24|     1.0|  53990.0|  48789.0|38790.0|   1|    157285|
|2021-04-02 11:48:16|    48|     1.0|  10699.0|   8450.0| 8990.0|   1|    113103|
|2021-04-02 11:48:59|    24|     1.0|  67199.0|  67199.0|55990.0|   1|    151375|
|2021-04-02 11:49:06|   180|     3.0|   2690.0|   2690.0| 2590.0|   1|    128134|
|2021-04-02 11:49:57|    24|     1.0|  96990.0|  72499.0|69990.0|   1|    152248|
|2021-04-02 11:50:59|    24|     1.0| 120390.0|   5990.0|13990.0|   1|    122363|
|2021-04-02 11:51:02|    26|     1.0|  19990.0|  13990.0|16999.0|   1|    126293|
|2021-04-02 11:5

In [6]:
concatenato.write.save("clicks_concatenato.csv",format="csv")